In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv(r'C:\Users\User\Car_Accidents.csv')

C:\Users\User\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [55]:
df.head()

,Unnamed: 0,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,COLLISION_ID,MM-DD,ON HOLIDAY,MM,Weekday,Time,Day Killed
0,0,2021-04-14 00:00:00,2021-07-24 05:32:00,unlisted,NaN,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,...,0,0,Following Too Closely,4407480,04-14,False,4,Wednesday,5,NaN
1,1,2021-04-13 00:00:00,2021-07-24 21:35:00,BROOKLYN,11217.0,40.68358,-73.97617,"(40.68358, -73.97617)",NaN,NaN,...,0,0,Unspecified,4407147,04-13,False,4,Tuesday,21,NaN
2,2,2021-04-15 00:00:00,2021-07-24 16:15:00,unlisted,NaN,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,...,0,0,Pavement Slippery,4407665,04-15,True,4,Thursday,16,NaN
3,3,2021-04-13 00:00:00,2021-07-24 16:00:00,BROOKLYN,11222.0,NaN,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,...,0,0,Following Too Closely,4407811,04-13,False,4,Tuesday,16,NaN
4,4,2021-04-12 00:00:00,2021-07-24 08:25:00,unlisted,NaN,0.00000,0.00000,"(0.0, 0.0)",EDSON AVENUE,NaN,...,0,0,Unspecified,4406885,04-12,False,4,Monday,8,NaN


In [56]:
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'])
df['COLLISION_ID']= df['COLLISION_ID'].astype('object')
df['MM-DD']= df['MM-DD'].astype('category')
df['MM']= df['MM'].astype('category')
df['Weekday']= df['Weekday'].astype('category')
df['DEATH OCCURED'] = np.where(df['NUMBER OF PERSONS KILLED'] > 0, 1, 0)
df['DEATH OCCURED']= df['DEATH OCCURED'].astype('bool')
df['Time'] = df['Time'].astype('category')
df['CONTRIBUTING FACTOR VEHICLE 1'].fillna('Unspecified', inplace =True)
df['DD'] =  df['MM-DD'].str.rsplit('-', n=1).str.get(1)

Creating additional features to give my model more data to predit from

In [57]:
df['DD'] =  df['MM-DD'].str.rsplit('-', n=1).str.get(1).astype('int64')
df['begin_of_month'] = (df['DD'] < 10).astype('uint8')
df['midddle_of_month'] = ((df['DD'] >= 10)&(df['DD'] < 20)).astype('uint8')
df['end_of_month'] = (df['DD'] >= 20).astype('uint8')
df['morning'] = df['Time'].map(lambda x: 1 if (x <= 11)& (x >= 7) else 0).astype('uint8')
df['day'] = df['Time'].map(lambda x: 1 if (x >= 12) & (x <= 18) else 0).astype('uint8')
df['evening'] = df['Time'].map(lambda x: 1 if (x >= 19) & (x <= 23) else 0).astype('uint8')
df['night'] = df['Time'].map(lambda x: 1 if (x >= 0) & (x <= 6) else 0).astype('int32')
df['winter'] = df['MM'].map(lambda x: x in [12, 1, 2]).astype('int32')
df['spring'] = df['MM'].map(lambda x: x in [3, 4, 5]).astype('int32')
df['summer'] = df['MM'].map(lambda x: x in [6, 7, 8]).astype('int32')
df['autumn'] = df['MM'].map(lambda x: x in [9, 10, 11]).astype('int32')


In [58]:
df.drop(columns=['LOCATION','Unnamed: 0','NUMBER OF PEDESTRIANS KILLED','NUMBER OF CYCLIST KILLED','NUMBER OF MOTORIST KILLED','NUMBER OF PERSONS KILLED','Day Killed'], inplace=True)

In [59]:
df.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID', 'MM-DD', 'ON HOLIDAY',
       'MM', 'Weekday', 'Time', 'DEATH OCCURED', 'DD', 'begin_of_month',
       'midddle_of_month', 'end_of_month', 'morning', 'day', 'evening',
       'night', 'winter', 'spring', 'summer', 'autumn'],
      dtype='object')

In [60]:
dfo = df[['BOROUGH','CONTRIBUTING FACTOR VEHICLE 1', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED','MM-DD', 'ON HOLIDAY','MM', 'Weekday', 'Time', 'DD',
       'begin_of_month', 'midddle_of_month', 'end_of_month', 'morning', 'day',
       'evening', 'night', 'winter', 'spring', 'summer', 'autumn']]
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1) 

In [61]:
df.head()

,CRASH DATE,CRASH TIME,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,COLLISION_ID,DEATH OCCURED,...,Time_14,Time_15,Time_16,Time_17,Time_18,Time_19,Time_20,Time_21,Time_22,Time_23
0,2021-04-14,2021-07-24 05:32:00,NaN,NaN,NaN,BRONX WHITESTONE BRIDGE,NaN,NaN,4407480,False,...,0,0,0,0,0,0,0,0,0,0
1,2021-04-13,2021-07-24 21:35:00,11217.0,40.68358,-73.97617,NaN,NaN,620 ATLANTIC AVENUE,4407147,False,...,0,0,0,0,0,0,0,1,0,0
2,2021-04-15,2021-07-24 16:15:00,NaN,NaN,NaN,HUTCHINSON RIVER PARKWAY,NaN,NaN,4407665,False,...,0,0,1,0,0,0,0,0,0,0
3,2021-04-13,2021-07-24 16:00:00,11222.0,NaN,NaN,VANDERVORT AVENUE,ANTHONY STREET,NaN,4407811,False,...,0,0,1,0,0,0,0,0,0,0
4,2021-04-12,2021-07-24 08:25:00,NaN,0.00000,0.00000,EDSON AVENUE,NaN,NaN,4406885,False,...,0,0,0,0,0,0,0,0,0,0


In [77]:
df.to_pickle(r'C:\Users\User\Car_Accidents_preprocessed.pkl')